In [20]:
import pixiedust
import numpy

In [9]:
%run ../visualisation.ipynb
%run ../read_graph.ipynb

g = read_nba_full("../../Data/Basketball/Players.csv",
                  "../../Data/Basketball/Edges.csv",
                  True, True)

all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
nba11_20 = g.subgraph(all11_20)

all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
nba15_20 = g.subgraph(all15_20)

all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
nba19_20 = g.subgraph(all19_20)

Reading full NBA graph, begin...
Reading file ../../Data/Basketball/Players.csv, begin...
Data lines read: 4800
Reading file ../../Data/Basketball/Edges.csv, begin...
Data lines read: 146369
Reading NBA vertices, begin...
Vertices read: 4800
Reading NBA edges, begin...
Weights loaded correctly
Edges read: 146369
Loading succesfull


In [10]:
def build_lpa(graph, output):
    result = graph.community_label_propagation()
    resultArray = result.membership
    i = 0
    for v in graph.vs:
        v["communities"] = [resultArray[i]]
        i += 1

    plot_full(graph, output, 1)

In [11]:
build_lpa(nba11_20, "all_stars_1120.svg")
build_lpa(nba15_20, "all_stars_1520.svg")
build_lpa(nba19_20, "all_stars_1920.svg")

Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful
Plotting graph with 60 vertices and 133 edges, begin...
Plotting sucessful
Plotting graph with 39 vertices and 38 edges, begin...
Plotting sucessful


In [12]:
def my_basic_lpa(graph, output):
    print("Basic lpa, begin...")
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [i]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < 20:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            com = v["communities"][0]
            counters[com] += 1
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_community = v["communities"][0]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    com = e.target_vertex["communities"][0]
                    counters[com] += 1
                elif v != e.source_vertex:
                    com = e.source_vertex["communities"][0]
                    counters[com] += 1
                    
            if sum(counters) != v.degree():
                print("Warn: sum = " + str(sum(counters)) + " degree = " + str(v.degree()))
                
            max_value = max(counters)
            max_indices = []
            
            for k in range(vertices_count):
                if (counters[k] == max_value):
                    max_indices.append(k)
            
            winner_idx = max_indices[random.randrange(len(max_indices))]
        
            if current_community in max_indices:
                winner_idx = current_community
                
            if winner_idx != current_community:
                convergence = False
                
            v["communities"] = [winner_idx]
            
    plot_full(graph, output, 1.5)

In [30]:
def my_basic_lpa_overlapping(graph, max_communities, output):
    print("Basic lpa with overlapping communities, begin...")
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [i]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < 20:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            for com in v ["communities"]:
                counters[com] += 1
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_communities = v["communities"]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    for com in e.target_vertex["communities"]:
                        counters[com] += 1
                elif v != e.source_vertex:
                    for com in e.target_vertex["communities"]:
                        counters[com] += 1
                    
            if sum(counters) != v.degree():
                print("Warn: sum = " + str(sum(counters)) + " degree = " + str(v.degree()))
                
            max_value = max(counters)
            sorted_counters = numpy.argsort(counters)[::-1]
            new_communities = []
            
            idx = 0
            while counters[sorted_counters[idx]] / max_value > 1 / max_communities:
                new_communities.append(sorted_counters[idx])     
                idx += 1
               
            if len(list(set(new_communities) - set(current_communities))) > 0:
                convergence = False
                
            v["communities"] = new_communities
            
    plot_full(graph, output, 1.5)

SyntaxError: invalid syntax (<ipython-input-30-59354edde725>, line 55)

In [29]:
my_basic_lpa(nba11_20, "all_stars_my_1120.svg")
my_basic_lpa(nba15_20, "all_stars_my_1520.svg")
my_basic_lpa(nba19_20, "all_stars_my_1920.svg")
my_basic_lpa_overlapping(nba11_20, 4, "all_stars_myo4_1120.svg")
my_basic_lpa_overlapping(nba15_20, 4, "all_stars_myo4_1520.svg")
my_basic_lpa_overlapping(nba19_20, 4, "all_stars_myo4_1920.svg")

Iteration 1,communities 83
Iteration 2,communities 21
Iteration 3,communities 10
Iteration 4,communities 5
Iteration 5,communities 5
Iteration 6,communities 3
Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful
Iteration 1,communities 60
Iteration 2,communities 17
Iteration 3,communities 12
Iteration 4,communities 9
Plotting graph with 60 vertices and 133 edges, begin...
Plotting sucessful
Iteration 1,communities 39
Iteration 2,communities 16
Plotting graph with 39 vertices and 38 edges, begin...
Plotting sucessful
Basic lpa with overlapping communities, begin...
Iteration 1,communities 83


KeyboardInterrupt: 

0.5